# Estimator design with state variables
The control law assumes that all the state variables are available for feed back. In most cases, not all the state variables are measured. The cost of the required sensors may be prohibitive, or it may be physically impossible to measure all of the state variables.
One method of estimating the state is to construct a full order model of the plant dynamics:

\begin{equation}
\dot{\hat{\pmb x}} = \pmb F \pmb{\hat x} + \pmb G u
\label{eq:xhat} 
\end{equation}

where $\pmb{\hat x}$ is the estimate of the actual state $\pmb x$. To study the dynamics of this estimator, we define the error in the estimate to be:

\begin{equation}
\tilde{\pmb x} = \pmb x - \pmb{\hat x}
\end{equation}

Then the dynamics of this error system are given by:

\begin{equation}
\dot{\tilde{\pmb x}} = \pmb F \tilde{\pmb x}, \space  \tilde{\pmb x}(0)= \pmb x(0) - \hat{\pmb x}(0)
\end{equation}

The error convergences to zero for a stable system (**F** stable), but we have no ability to influence the rate at which the state estimate converges to the true state. Furthermore, the error is converging to zero at the same rate as the natural dynamics of **F**. If this convergence rate were satisfactory, no control or estimation would be required. The golden rule is: when in trouble use feedback. Consider the feedback the difference between the measured and estimate outputs and correcting the model continuously with this error signal. The equation for this scheme is: 

\begin{equation}
\dot{\hat{\pmb x}} = \pmb F \hat{\pmb x} + \pmb G u + \pmb L(y - \pmb H \hat{\pmb x})
\label{eq:dot_hat_x} 
\end{equation}

Here **L** is a proportional gain defined as:
\begin{equation}
\pmb L = [l_1,l_2,.....,l_n]^T
\end{equation}

and is chosen to achieve satisfactory error characteristics. The dynamics of the error can be obtained by subtracting the estimate $\eqref{eq:dot_hat_x}$ from $\eqref{eq:stat-form}$ to get the error equation:

\begin{equation}
    \dot{\tilde{\pmb x}} = (\pmb F - \pmb LH)\tilde{\pmb x}
\end{equation}

and the characteristic equation of the error is now given by:

\begin{equation}
    det[s \pmb I - (\pmb{F - LH})] = 0
    \label{eq:det_est} 
\end{equation}

If we can choose $\pmb L$ so that $\pmb F - \pmb{LH}$ has stable and reasonably fast eigenvalues, then $\tilde{\pmb x}$ will decay to zero and remain there independent of the known forcing function $u(t)$ and its effect on the state $x(t)$ and irrespective on the initial condition $\tilde{\pmb x}(0)$. This means that $\hat{\pmb x(t)}$ will converge to $\pmb{x}(t)$, regardless of the value of $\hat{\pmb x}(0)$. However we can typically choose $\pmb L$ so that the error system is still at least stable and the error remains acceptably small, even with modeling errors and disturbing inputs. The selection of $\pmb L$ can be approached in exactly the same fashion as $\pmb K$ is selected in the control law design. If we specify the desired location of the estimator error poles as:

\begin{equation}
    s_i = \beta_1, \beta_2,....\beta_n
\end{equation}

then the desired estimator characteristic equation is:

\begin{equation}
    \alpha_e(s) = (s-\beta_1)(s-\beta_2)....(s-\beta_n)
    \label{eq:char_eq} 
\end{equation}

We can then solve for $\pmb L$ by comparing coefficients in Eqs. $\eqref{eq:det_est}$ and $\eqref{eq:char_eq}$.

## The observability
As begun in the introduction it's time to spent some raw talking about the **observability** of state variables. In a development exactly parallel with the control-law case, we can find a transformation to take a given system to observer canonical form if and only if the system has structural properties that in this case we call as mention just like above. Roughly speaking, observability refers to our ability to deduce information about all the modes of the system by monitoring only the sensed outputs. Not observability results when some mode or subsystem is disconnected physically from the output and therefore no longer appears in the measurements. The mathematical test for determining observability is that the **observability matrix**:  

\begin{equation}
\mathcal{O} = \begin{bmatrix}
\pmb H\\
\pmb{HF}\\
..\\
..\\
\pmb{HF}^{n-1}
\end{bmatrix}
\label{eq:obs_mat}
\end{equation}

must have independent columns. In general, we can find a transformation to observer canonical form if and only if the observability matrix is nonsingular (full rank).Note that is analogous to our earlier conclusion for transforming system matrices to control canonical form.
As with control law design, we could find the transformation to observer form, compute the gains from the equivalent of $\pmb F_o - \pmb L \pmb H_o$ and transform back. An alternative method of computing $\pmb L$ is to use the Ackermann's formula in estimator form: 

\begin{equation}
\pmb L = \alpha_e(\pmb F)\mathcal{O}^{-1}
\begin{bmatrix}
0 \\
0 \\
.. \\
.. \\
1 \\
\end{bmatrix} 
\end{equation}

where $\mathcal{O}$ is the observability matrix given in $\eqref{eq:obs_mat}$.

### Hand-on labs of observability of the cart-pendulum
This code snippet goes to calculate the observability matrix when the measurement matrix **H** (in the code is C) select a sensor related to x cart position and then reply the calculation for $\theta$ measurement. It can be highlighted that the determinant is nonzero (full rank) in the first trial and zero in the second one. In particular  tuns out that in the second is impossible to reconstruct the x position because that coordinate does not affect the others; equations of cart pendulum are invariant for translations in x position. It's important reconstruct the cart position to have full estimate of state variable putting them in the LQR block for resulting compensator placed in the feed back (as it will see later).

```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];

C = [1 0 0 0]; 

obsv(A,C)
det(obsv(A,C))

%%  only observable if x measured... because x can't be
%% reconstructed
C = [0 0 1 0];

obsv(A,C)
det(obsv(A,C))
```
The terminal result:
```
ans =

    1.0000         0         0         0
         0    1.0000         0         0
         0   -0.2000    2.0000         0
         0    0.0400   -0.4000    2.0000


ans =

     4


ans =

         0         0    1.0000         0
         0         0         0    1.0000
         0    0.1000   -6.0000         0
         0   -0.0200    0.2000   -6.0000


ans =

     0
```
---

## Estimator poles selection. The optimal full-state estimation 
In comparison with the selection of controllers poles, estimator poles selection require to be concerned with a much different relationship than with control effort. As in controller, there is a feedback term in the estimator that grows in magnitude as the requested speed of response increases. However this feedback is in the form of an electronic signal or a digital word in a computer, so its growth causes no special difficulty. The important consequence of increasing the speed of response of an estimator is that the bandwidth of the estimator becomes higher, thus causing more sensor noise to pass on to the control actuator. Like the controller the estimator design (and the poles selection) is a balance between good transient response and low-enough bandwidth that sensor noise does not significantly impact the actuator activity.   
When deriving the optimal full-state estimator, it is necessary to re-introduce disturbances to the state, $\pmb w_d$ , and sensor noise, $\pmb w_n$ :

\begin{align}\label{eq:augsys} 
\dot{\pmb x} &= \pmb{Fx} + \pmb{Gu} + \pmb{w}_d\\
\pmb y &= \pmb Hx + \pmb Ju + \pmb{w}_n 
\end{align}

By hypothesis is assumed that disturbance and noise are zero-mean Gaussian processes with known covariances:

\begin{align}
E(\pmb{w}_d(t) \pmb{w}_d(\tau)^*) &= Q \delta (t - \tau)\\
E(\pmb{w}_n(t) \pmb{w}_n(\tau)^*) &= R \delta (t - \tau)
\end{align}

Here $E$ is the expected value and $\delta (.)$ is the Dirac delta function. The matrices $Q$ and $R$ are positive semi-definite with entries containing the covariances of the disturbance and noise terms. It is possible to obtain an estimate $\hat{\pmb x}$ of the full-state $\pmb x$ from measurements of the input $\pmb u$ and output $\pmb y$, via the following estimator dynamical system:

\begin{align}\label{eq:estimator}
\dot{\hat{\pmb x}} &= \pmb F \hat{\pmb x} + \pmb Gu + \pmb L (\pmb y - \hat{\pmb y})\\
\hat{\pmb y} &= \pmb H \hat{\pmb x} + \pmb Ju
\end{align}

The matrices $\pmb F, \pmb G, \pmb H, \pmb J$ are obtained from the system model and the filter
gain $\pmb L$ is determined via a similar procedure as in LQR. $\pmb L$ is given by:

\begin{equation}
\pmb L = \pmb Y \pmb H^* \pmb R
\end{equation}

where $\pmb Y$ is the solution to another algebraic Riccati equation:

\begin{equation}
    \pmb YF^* + \pmb FY - \pmb{YH}^* \pmb R^{-1} \pmb H \pmb Y + \pmb Q = \pmb 0
\end{equation}

This solution is commonly referred to as the Kalman filter, and it is the optimal full-state estimator with respect to the following cost function:

\begin{equation}
    J = \lim\limits_{t \to \infty} {E[\tilde x(t)^* \tilde x(t)]}
\end{equation}

This cost function implicitly includes the effects of disturbance and noise, which are required to determine the optimal balance between aggressive estimation and noise attenuation. Thus, the Kalman filter is referred to as linear quadratic estimation (LQE), and has a dual formulation to the LQR optimization.
The estimator dynamical system is expressed in terms of the estimate $\hat{\pmb x}$ with inputs $\pmb y$ and $\pmb u$. If the system is observable it is possible to place the eigenvalues of $\pmb F - \pmb L \pmb H$ arbitrarily with choice of $\pmb L$ . When the eigenvalues of the estimator are stable, then the state estimate $\hat{\pmb x}$ converges to the full-state $\pmb x$ asymptotically, as long as the model faithfully captures the true system dynamics. To see this convergence, consider the dynamics of the estimation error $\pmb{\epsilon} = \pmb x - \pmb{\hat x}$:

\begin{align}
\dot{\pmb{\epsilon}} &= {\pmb{x}} - \hat{\pmb{x}} \nonumber\\
                     &= [\pmb{Fx} + \pmb{Gu} + \pmb{w}_d] - [(\pmb F - \pmb{LH}) \hat{\pmb x} + \pmb{Ly} + (\pmb G - \pmb{LJ})\pmb u]\\
                     &= (\pmb F - \pmb{LH}) \epsilon + \pmb w_d - \pmb L \pmb w_n \nonumber
\end{align}

Therefore, the estimate $\hat{\pmb x}$ will converge to the true full state when $\pmb F - \pmb L \pmb H$ has stable eigenvalues. As with LQR, there is a trade-off between over-stabilization of these eigenvalues and the amplification of sensor noise. This is similar to the behavior of an inexperienced driver who may hold the steering wheel too tightly and will overreact to every minor bump and disturbance on the road.

### Plant disturbance and Noise measurement. Hand-on labs
To give a demostration of best choise of $L \equiv K_f$ it's considered a system with $J=0$ in $\eqref{eq:augsys}$ and pendulum in bottom position. It's better create a practical solution around a stable equilibrium point because the algebra is the same. The system represented in the <a href=#fig5>picture</a> shows the linearized system with input $u$ to be the force applied to the cart pendulum, $w_d$ the gaussian zero-mean disturbance applied to the model and the cart position ($\pmb H=[1 0 0 0]$) noised as output of state variable observed. The goal is reconstruct the full state estimate. To do it is necessary to re-represent the system like 2 *augmented* blocks to feed in Matlab functions:     

\begin{align}
\pmb Q &= 0.1 \pmb{I} & \pmb x = \begin{bmatrix} x\\ \dot x\\ \theta\\      \dot{\theta} \end{bmatrix}\\
R &= 1   
\end{align}

\begin{equation} \label{eq:aug-eqs}
\begin{split}
\pmb{G}_A & = \begin{bmatrix}\pmb G & \pmb Q & \pmb{\underline{0}}x \pmb B\end{bmatrix}\\
\pmb{F}_A & = \pmb{F}\\
\pmb{H}_A & = \pmb H\\
\pmb{J}_A & = \begin{bmatrix} 0 & 0 & 0 & 0 & 0 & R \end{bmatrix}
\end{split}
\end{equation}

---

|       | 
|:-------:|
|![fig5](images/kfscheme.png)|

<p><a name="fig11">Fig 11</a> A scheme of Kalman Filter; d and n are disturbance of state system and noise measurement introduced in the simulation</p>

According to the equations $\eqref{eq:estimator}$, by inspecting $\eqref{eq:stat-form}$ and $\eqref{eq:output}$ the *new* estimator-system matrices are:

\begin{equation}
\label{eq:kalman-system}
\begin{split}
\pmb{F}_k &= \pmb F - \pmb{K}_f \pmb H\\
\pmb{G}_k &= \begin{bmatrix}\pmb G \pmb{K}_f \end{bmatrix}\\
\pmb{H}_k &= \pmb I\\
\pmb{J}_k &= \pmb{0}
\end{split}
\end{equation}

The input $u$ is augmented also to taking into account of disturbance plant and noise measurement:

\begin{equation}
\label{eq:u_noise}
\pmb{u}_{aug} = \begin{bmatrix} u & {Q}^2 * \pmb{u}_{dist} & u_{noise}\end{bmatrix}
\end{equation}

The code:
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

% y = [x; dx; theta; dtheta];
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

H = [1 0 0 0];  

J = zeros(size(H,1),size(G,2));

%%  Augment system with disturbances and noise
Q = .1*eye(4);  % disturbance covariance
R = 1;       % noise covariance

GA = [G Q 0*G];  % augment inputs to include disturbance and noise

sysH = ss(F,GA,H,[0 0 0 0 0 R]);  % build big state space system... with single output

% system with full state output, disturbance, no noise
sysFullOutput = ss(F,GA,eye(4),zeros(4,size(GA,2)));  

%%  Build Kalman filter
[L,P,E] = lqe(F,Q,H,Q,R);  % design Kalman filter
Kf = (lqr(F',H',Q,R))';   % alternatively, possible to design using "LQR" code

sysKF = ss(F-L*H,[G L],eye(4),0*[G L]);  % Kalman filter estimator

%%  Estimate linearized system in "down" position (Gantry crane)
dt = .01;
t = dt:dt:50;

uDIST = randn(4,size(t,2));
uNOISE = randn(size(t));
u = 0*t;
u(100:120) = 100;     % impulse
u(1500:1520) = -100;  % impulse

uAUG = [u; Q*Q*uDIST; uNOISE];

[y,t] = lsim(sysH,uAUG,t);
[xtrue,t] = lsim(sysFullOutput,uAUG,t);


[x,t] = lsim(sysKF,[u; y'],t);

plot(t,xtrue,'-',t,x,'--','LineWidth',2)

figure
plot(t,y)
hold on
plot(t,xtrue(:,1),'r')
plot(t,x(:,1),'k--')
```
---

|         |
|:---------:|
|![graph](images/cartpendxpos.png)|
<p> <a name="simnoise"> Fig. 12 </a> Simulation of a noise position from an impulse system. Estimate position "cover" the true</p>

|         |
|:---------:|
|![graph-est](images/true-estimate.png)|
<p> <a name="true-estimate"> Fig. 13 </a> Showing the state variable "true" and estimate</p>

With the estimate state, it's the time to "*feedback*" it in a Linear Quadratic Regulator to combine control and estimation together. 

## Optimal sensor-based control: Linear quadratic Gaussian (LQG)
The full-state estimate from the Kalman filter is generally used in conjunction
with the full-state feedback control law from LQR, resulting in optimal sensor-
based feedback. Remarkably, the LQR gain $K_r$ and the Kalman filter gain $K_f$
may be designed separately, and the resulting sensor-based feedback will re-
main optimal and retain the closed-loop eigenvalues when combined.
Combining the LQR full-state feedback with the Kalman filter full-state estimator results in the linear-quadratic Gaussian (LQG) controller. The LQG controller is a dynamical system with input y, output u, and internal state $\hat{\pmb x}$:

\begin{equation} \label{eq:LQGsys}
\begin{split}
\frac{d}{dt}\hat{\pmb x} =& (\pmb F - \pmb{K}_f \pmb H - \pmb G \pmb{K}_r) \hat{\pmb x} + \pmb{K}_f \pmb y\\
\pmb u =& -\pmb K_r \hat{\pmb x} 
\end{split}
\end{equation}

The LQG controller is optimal with respect to the following ensemble-averaged
version of the cost function from $\eqref{eq:costfunc}$:

\begin{equation}
\label{eq:costfunc2}
J(t) = \left\langle 
\int\limits_0^{t} [\pmb x(\tau)^* \pmb Q \pmb x(\tau) + \pmb u(\tau)^* \pmb R \pmb u(\tau)  d\tau]  
\right\rangle.
\end{equation}

The controller $\pmb u = \pmb K_r \hat{\pmb x}$ is in terms of the state estimate, and so this cost
function must be averaged over many realizations of the disturbance and noise.
Applying LQR to $\hat{\pmb x}$ results in the following state dynamics:

\begin{equation}
\begin{split}
\frac{d}{dt} \pmb x =& \pmb F \pmb x - \pmb G \pmb K_r \hat{\pmb x} + \pmb w_d\\
                    =& \pmb F \pmb x - \pmb G \pmb K_r {\pmb x} + \pmb G \pmb K_r(\pmb{x-\hat x}) + \pmb w_d\\
                    =& \pmb F \pmb x - \pmb G \pmb K_r {\pmb x} + \pmb G \pmb K_r \pmb \epsilon + \pmb w_d
\end{split}
\end{equation}

Again $\pmb \epsilon = \pmb{x - \hat x}$ as before. Finally the closed-loop system may be written as

\begin{align}
\label{eq:clsys}
\frac{d}{dt}
\begin{bmatrix} 
\pmb x\\
\pmb \epsilon   
\end{bmatrix} = 
\begin{bmatrix}
\pmb F - \pmb G \pmb K_r & \pmb{GK}_r\\
\pmb 0 & \pmb F - \pmb K_f \pmb H
\end{bmatrix}
\begin{bmatrix}
\pmb x\\
\pmb \epsilon 
\end{bmatrix}
+
\begin{bmatrix}
\pmb I & \pmb 0\\
\pmb I & -\pmb K_f
\end{bmatrix}
\begin{bmatrix}
\pmb w_d\\
\pmb w_n
\end{bmatrix}
\end{align}


Thus, the closed-loop eigenvalues of the LQG regulated system are given by
the **eigenvalues** of $\pmb F - \pmb{GK}_r$ and $\pmb F - \pmb K_f \pmb H$, which were optimally chosen by the
LQR and Kalman filter gain matrices, respectively. This in control theory is called **the separation principle** which essentially means it's possible design the LQR and LQE separately and when they are combined together the system retains the desirable properties of each of them.  

|           |
|:---------:|
|![Here LQG](images/LQGblock.png)|

<p><a name="LQG">Fig 14</a> Schematic illustrating the linear quadratic Gaussian (LQG) controller for optimal closed-loop feedback based on noisy measurements y. The optimal LQR and Kalman filter gain matrices $K_r$ and $K_f$ may be designed independently, based on two different algebraic Riccati equations. When combined,the resulting sensor-based feedback remains optimal.</p>

It's worthwhile to mention that LQG framework relies on an accurate model of the system and knowledge of the magnitudes of the disturbances and measurement noise, which are assumed to be Gaussian processes. In real-world
systems, each of these assumptions may be invalid, and even small time delays and model uncertainty may destroy the robustness of LQG and result in instability. The lack of robustness of LQG regulators to model uncertainty
motivates the introduction of *robust control*. For example, it is possible to robustify LQG regulators through a process known as *loop-transfer recovery*. However, despite robustness issues, LQG control is extremely effective
for many systems, and is among the most common control paradigms.

### Apply LQG to put all togheter. Hand-on lab

To apply an LQG regulator to the inverted pendulum on a cart, we will simulate the full nonlinear system in Simulink, as shown in <a href="#LQG">figure 15</a>.
The non-linear cart pendulum is connected by the force $u$ in addiction to a gaussian random noise disturbance that affected the model. The output of the cart-pendulum is the state-variable vector connected with $\pmb H$ matrix to go input in the estimator block only with the position-cart sensor noise measurement gaussian distributed also; the relation is $y = \pmb H \pmb x + w_n$. The Kalman filter reconstruct the full state variable for the compensator block.  

|           |
|:---------:|
|![fig9](images/LQG.png)|

<p><a id="LQG">Fig. 15 </a> Matlab Simulink model for sensor-based LQG feedback control.</p>

The system starts near the vertical equilibrium, at $\pmb x_0 = [0, 0, 3.14, 0]$, and **it's commanded a step** in the cart position from x = 0 to x = 1 at t = 10. The resulting response is shown in <a href="#stateLQG">figure 16</a>. Despite noisy measurements (<a href="#positionLQG">figure 17</a>) and disturbances (<a href="#noiseLQG">figure 18</a>), the controller is able to effectively track the reference cart position while stabilizing the inverted pendulum.

|           |
|:---------:|
|![figure10](images/stateLQG.png)|
<p><a id="stateLQG">Fig. 16</a> Output response using LQG feedback control.</p>

|           |
|:---------:|
|![figure11](images/positionLQG.png)|
<p><a id="positionLQG">Fig. 17</a> Noisy measurement used for the Kalman filter, along with the underlying noiseless signal and the Kalman filter estimate.</p>

|           |
|:---------:|
|![figure12](images/noiseLQG.png)|
<p><a id="noiseLQG">Fig. 18</a> Noisy measurement used for the Kalman filter, along with the underlying noiseless signal and the Kalman filter estimate</p>

The m-code set some values needed for the simulation
```
clear all, close all, clc

%% Set cart-pendulum parameters
m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)
%% Base system Matrices
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

% Set cart-position
H = [1 0 0 0];  

J = zeros(size(H,1),size(G,2));

%% Linear Quadratic Regulator
Q = [1 0 0 0;
    0 1 0 0;
    0 0 1 0;
    0 0 0 1];
R = .000001;
K = lqr(F,G,Q,R);  % design controller u = -K*x


%% Augmented system
%  Augment system with disturbances and noise
Vdmag = .04;
Vd = Vdmag*eye(4);  % disturbance covariance
Vn = .0002;       % noise covariance

% GF sysC sysFulloutput not used in Simulink
GF = [G Vd 0*G];  % augment inputs to include disturbance and noise
sysH = ss(F,GF,H,[0 0 0 0 0 Vn]);  % build big state space system... with single output
 % system with full state output, disturbance, no noise
sysFullOutput = ss(F,GF,eye(4),zeros(4,size(GF,2))); 

%  Build Kalman filter 2 alternatives
[L,P,E] = lqe(F,eye(4),H,Vd,Vn);  % design Kalman filter
Kf = (lqr(F',H',Vd,Vn))';   % alternatively, possible to design using "LQR" code

% The system matrices for LQE. Used in Simulink
sysKF = ss(F-Kf*H,[G Kf],eye(4),0*[G Kf]);  % Kalman filter estimator

```

Here the m-code to print the result
```
CC = [0    0.4470    0.7410
    0.8500    0.3250    0.0980
    0.9290    0.6940    0.1250
    0.4940    0.1840    0.5560
    0.4660    0.6740    0.1880
    0.3010    0.7450    0.9330
    0.6350    0.0780    0.1840];

figure(1)
plot(yn.Time,yn.Data(:,2),'Color',[.5 .5 .5])
hold on
plot(x.Time,x.Data(:,1),'Color',[0 0 0],'LineWidth',2)
plot(yKF.Time,yKF.Data(:,1),'--','Color',CC(1,:),'LineWidth',2)
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('Measurement')
l1 = legend('y (measured)','y (no noise)','y (KF estimate)');
set(l1,'Location','SouthEast')
grid on

%%
figure(2)
subplot(3,1,1)
plot(wd.Time(1:100:end),wd.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
ylabel('Disturbance, d')
subplot(3,1,2)
plot(wn.Time(1:100:end),wn.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
ylabel('Noise measurement, n')
subplot(3,1,3)
plot(yn.Time(1:100:end),yn.Data(1:100:end,1),'LineWidth',1.2)
xlim([45 50])
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('Actuation, u')
grid on

%%
figure(3)
plot(x.Time,x.Data,'LineWidth',2)
l1 = legend('x','v','\theta','\omega')
set(gcf,'Position',[100 100 500 200])
xlabel('Time')
ylabel('State')
grid on

```